In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from bs4 import BeautifulSoup
import warnings


c:\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
files = ['Youtube01-Psy.csv','Youtube02-KatyPerry.csv','Youtube03-LMFAO.csv','Youtube04-Eminem.csv','Youtube05-Shakira.csv']
frames = [pd.read_csv(f) for f in files]
data = pd.concat(frames,axis=0,ignore_index=True)
print(data.shape)
data.head()

(1956, 5)


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [10]:
data = data.drop(['COMMENT_ID','AUTHOR','DATE'],axis=1)
print(data.shape)
data.head(n=10)

(1956, 2)


,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1
5,"Hey, check out my new website!! This site is a...",1
6,Subscribe to my channel ﻿,1
7,i turned it on mute as soon is i came on i jus...,0
8,You should check my channel for Funny VIDEOS!!﻿,1
9,and u should.d check my channel and tell me wh...,1


In [11]:
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
data['CONTENT'] = data['CONTENT'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text())

In [12]:
print(data.shape)
for idx,x in data.iterrows():
    if x['CONTENT'].find('Hey guys,')!=-1:
        print(str(idx),end='\n')
        print("-->" + str(x['CLASS']),end="\n")
        print("-->" + x['CONTENT'],end="\n\n")

(1956, 2)
569
-->1
-->Hey guys, I was working last night on this project, it's a music streamer  like spotify, but it allows you to download the files to listen to when  you're offline. Opening it as a free beta, feel free to try it out :) download: https://mega.co.nz/#!ehVjzKyA!5bGKg2iWPHZOHWAEUesrWnegqG_lABcO7Rw9WFX8hAo﻿

702
-->1
-->Hey guys, I'm a human.But I don't want to be a human, I want to be a sexy fucking giraffe.I already have the money for the surgery to elongate my spinal core, the surgery to change my skin pigment, and everything else! Like this post so others can root me on in my dream!!!!Im fucking with you, I make music, check out my first song! #giraffebruuh﻿



In [13]:
import re,string
data['CONTENT'] = data['CONTENT'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
data.head()

,CONTENT,CLASS
0,Huh anyway check out this youtube channel koby...,1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdevcom,1
3,me shaking my sexy ass on my channel enjoy ﻿,1
4,watchvvtaRGgvGtWQ Check this out ﻿,1


In [14]:
data.groupby('CLASS').count()

,CONTENT
CLASS,
0,951
1,1005


In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
words = set(stopwords.words('english'))
def stop_rem(s):
    new_s = ''
    word_list = word_tokenize(s)
    for w in word_list :
        if w.lower() not in words and isEnglish(w):
            new_s = new_s + w.lower() + ' '
    return new_s
data['CONTENT'] = data['CONTENT'].apply(lambda x: stop_rem(x))

In [17]:
data.head()

,CONTENT,CLASS
0,huh anyway check youtube channel kobyoshi02,1
1,hey guys check new channel first vid us monkey...,1
2,test say murdevcom,1
3,shaking sexy ass channel enjoy,1
4,watchvvtarggvgtwq check,1


In [34]:
from collections import Counter
token = []
for idx,x in data.iterrows():
    token = token + word_tokenize(x['CONTENT'])
print(len(token))
token = ["_link_feature_" if x.find('http')!=-1 else x for x in token]
token = ["_link_feature_" if x.find('watch')!=-1 and len(x)>9 else x for x in token]
c= Counter(token)
print(c)

15624
Counter({'check': 559, 'video': 294, 'like': 234, 'please': 231, 'song': 231, 'subscribe': 209, 'love': 189, 'channel': 173, 'im': 157, 'music': 144, 'new': 122, '_link_feature_': 118, 'guys': 114, 'get': 97, 'dont': 92, 'make': 87, 'hey': 86, 'money': 86, 'views': 84, 'people': 84, 'youtube': 82, 'best': 78, 'katy': 75, 'comment': 73, 'know': 71, 'would': 67, 'videos': 66, 'see': 65, 'good': 64, 'thank': 63, 'u': 62, 'really': 60, 'one': 58, '2': 58, 'thanks': 57, 'go': 57, 'us': 55, 'time': 53, 'old': 53, 'could': 53, 'help': 52, 'much': 52, 'everyone': 51, 'world': 50, 'called': 49, 'visit': 49, 'eminem': 49, 'want': 48, 'thumbs': 47, 'billion': 46, 'perry': 45, 'give': 45, 'free': 43, 'songs': 43, 'mixtape': 41, 'still': 41, 'today': 40, 'share': 40, 'watch': 40, 'way': 39, 'chance': 39, 'playlist': 38, 'take': 38, 'website': 37, '1': 37, 'home': 36, 'working': 36, 'rapper': 36, 'moneygqcom': 36, 'making': 36, 'online': 35, 'watching': 34, 'year': 34, 'back': 34, 'nice': 33, 